# Welcome to the CARTO Python SDK workshop!!

This is a self-contained tutorial for educational purposes on how to use the CARTO Python SDK to work with the CARTO APIs

## [The Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/latest/)

For this workshop we will use [The Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/latest/), a web application that allows executing live Python code (among others) along with equations, graphs, explanatory text, etc.

You don't need to install anything for the workshop, just play around with **your** notebook.

In case you want to install additional libraries, you still can use the `pip` command from your notebook:

`!pip install carto==1.0.1`

_But be aware that the server is shared among all the attendees to the workshop and bad things may happen._

The mechanics of a Jupyter Notebook are very intuitive:

- The notebook is divided in cells
- Just **click** over cells and run them (**Shift+Enter**) to execute their code
- You have to run each cell sequentially but at any moment you can go to a previous cell and re-run it.
- You only can run one cell at a time so if a cell runs forever, stop it and review the code, you may have introduced a `bug`.
- Once a cell has been run, **double click** it to edit it and be able to re-run it.

**You can use the shortcut `Shift+Enter` to run the current cell and jump to the next one.**

### A Markdown cell

We are going to use two different types of cells: `Markdown` and `Code`

This is a [Markdown](https://daringfireball.net/projects/markdown/syntax) cell and it is for writing text with marks.

###### This is a title

This is a list:

*   Red
*   Green
*   Blue

**bolds** _italics_ [links](https://carto.com) and inline images ![images](https://cartodb-libs.global.ssl.fastly.net/cartodb.com/files/img/layout/gallery/trees/thumbnail.b793e4f7.png)

In [ ]:
# This a `Code` cell. Since this is a `Python` notebook you can run only `Python` code

print('Hello world!')

In [ ]:
# <<<<< See the * while running this cell. The * means that the cell is running, be aware when you run cells infinitelly
# This is an Infinite loop. The cell will run forever and will make other cells to stop working.
# To finish the execution you have to stop this cell manually, by clicking on the cell and then the stop button in the top bar
from time import sleep
while True:
    sleep(5)

## [CARTO Python SDK documentation](http://carto-python.readthedocs.io/en/latest/)

You can find the latest documentation about the `carto-python` library [here](http://carto-python.readthedocs.io/en/latest/), please keep this document open for further reference during the workshop.

A part from the docs, the `carto-python` library is free and the source coude is available in [this Github repository](https://github.com/CartoDB/carto-python).

If during the workshop you find anything that does not work as expected or you have a suggestion or new feature that is missing, don't be shy and report it [here](https://github.com/CartoDB/carto-python/issues/new), we'll be glad to improve the `carto-python` for your needs.

## Local [Installation](http://carto-python.readthedocs.io/en/latest/installation.html#)

**You don't need to do this for the workshop** but when working on your own projects you should install `carto-python` on your local machine.

The recommended Python environment is:

- Python 2.7+ or Python 3.0+
- [pip](https://pip.pypa.io/en/stable/installing/) as a package manager
- (optionally) [virtualenv](https://virtualenv.pypa.io/en/stable/installation/) for isolated development environments

How to install `carto-python` locally:

- Create a new Python virtual environment (optional):

```
virtualenv env
source env/bin/activate
```

- Install carto

```
pip install carto
```

And you are ready to start coding!!

## Quickstart

Let's run our first Python script with the Carto Python SDK!

In [ ]:
# Let's import some system packages
import os
import sys
import logging
import warnings
from pprint import pprint as _print

# We don't want warnings during the workshop (but you should take care of them in production)
logging.captureWarnings(True)
warnings.filterwarnings('ignore')

### Setup your user credentials

In [ ]:
# FILL UP YOUR CREDENTIALS!!
USERNAME = 'your_user_name'
# WARNING: Don't share your API_KEY, you should always keep it private in your projects.
API_KEY = 'your_api_key'

BASE_URL = "https://{user}.carto.com/".format(user=USERNAME)

In [ ]:
# Let's import the classes we are using
from carto.auth import APIKeyAuthClient
from carto.exceptions import CartoException
from carto.sql import SQLClient

# Requests are authenticated, so we create a new APIKeyAuthClient with your credentials
auth_client = APIKeyAuthClient(BASE_URL, API_KEY)

# SQLClient is used to send SQL queries
sql = SQLClient(auth_client)

# let's send a simple SQL query
try:
    data = sql.send('select version()')
except CartoException as e:
    print("some error ocurred", e)

# now we have the data dictionary, let's see what it has
print data

In [ ]:
# you can get the results from the rows key
print data["rows"]

In [ ]:
# this function iterates over the result from the SQL API and prints the data
def print_result(result):
    print("====================")
    if result is None or result["rows"] is None:
        print("no data to print")
        return
        
    counter = 0
    for row in result["rows"]:
        for field in result["fields"]:
            print(field + "(" + result["fields"][field]["type"] + "): " + str(result["rows"][counter][field]))
        counter += 1
        print("--------------------")
    print("====================")
        
print_result(data)

## Exceptions

All errors while using the `carto-python` library are wrapped into a `CartoException` object. Always use `try-except` clauses when calling to `carto-python` functions.

Let's write a wrong SQL query, that will throw an error.

In [ ]:
# A wrong SQL query throws a CartoException
try:
    data = sql.send('select version_fuckkk()')
except CartoException as e:
    print("some error ocurred", e.message)

## CARTO APIs

The CARTO Python SDK implements the official CARTO APIs.

- [Import API](https://carto.com/docs/carto-engine/import-api)
- [SQL API](https://carto.com/docs/carto-engine/sql-api)
- [Maps API](https://carto.com/docs/carto-engine/maps-api)

Additionally, you have access to other [non-public APIs](http://carto-python.readthedocs.io/en/latest/non_public_apis.html) but be aware that those APIs may change without prior notice, so its usage is not recommended

Let's walk through the different APIs.

### Import API

You can import local or remote datasets into CARTO via the Import API in several ways.

#### FileImport

In [ ]:
# << Wait until the * stops
from carto.file_import import FileImportJobManager
import time

# write here the URL for the dataset or the path to a local file (local to the server...)
# take a look at the supported data formats -> https://carto.com/docs/carto-engine/import-api/importing-geospatial-data#supported-geospatial-data-formats
LOCAL_FILE_OR_URL = "https://academy.cartodb.com/d/tornadoes.zip"

file_import_manager = FileImportJobManager(auth_client)
file_import = file_import_manager.create(LOCAL_FILE_OR_URL)

# return the id of the import
file_id = file_import.get_id()

file_import.run()

# the import API is asyncrhonous, so once sent the file, check periodically the status of the job to know when it's finished
while(file_import.state != "complete" and file_import.state != "created"
            and file_import.state != "success"):
    time.sleep(5)
    file_import.refresh()
    if (file_import.state == 'failure'):
        print('The error code is: ' + str(file_import))
        break
        
print("`tornados` dataset successfully created!")

#### List current file import jobs

In [ ]:
from carto.file_import import FileImportJobManager

file_import_manager = FileImportJobManager(auth_client)
file_imports = file_import_manager.all()

if len(file_imports) == 0:
    print("You don't have any file import jobs")
else:
    print(file_imports)

#### Sync datasets

In [ ]:
# << Wait until the * stops
from carto.sync_tables import SyncTableJobManager
import time

# how often to sync the dataset (in seconds)
SYNC_TIME = 900
# write here the URL for the dataset to sync
URL_TO_DATASET = "https://academy.cartodb.com/d/tornadoes.zip"

sync_table_manager = SyncTableJobManager(auth_client)
sync_table = sync_table_manager.create(URL_TO_DATASET, SYNC_TIME)

# return the id of the sync
sync_id = sync_table.get_id()
sync_table.run()

# the import API is asyncrhonous, so once sent the file, check periodically the status of the job to know when it's finished
while(sync_table.state != "complete" and sync_table.state != "created"
            and sync_table.state != "success"):
    time.sleep(5)
    sync_table.refresh()
    if (syncTable.state == 'failure'):
        print('The error code is: ' + str(sync_table.error_code))
        print('The error message is: ' + str(sync_table.error_message))
        break

# force sync
sync_table.refresh()
sync_table.force_sync()

print("Sync dataset created. Check your CARTO datasets dashboard.")

### SQL API

In [ ]:
from carto.sql import SQLClient
from carto.exceptions import CartoException

# Requests are authenticated, so we create a new APIKeyAuthClient with your credentials
auth_client = APIKeyAuthClient(BASE_URL, API_KEY)

sql = SQLClient(auth_client)

try:
    data = sql.send('select * from tornados limit 10;')
except CartoException as e:
    print("some error ocurred", e)

print_result(data)

#### Export as CSV

In [ ]:
query = "select * from tornados limit 10"
# You can export data from CARTO in many different formats. Check the documentation to know more
result = sql.send(query, format='csv')

print(result)

filename = "tornados_" + USERNAME + ".csv"
with open(filename, 'w') as f:
    f.write(result)
f.close()

### Spatial analysis through the SQL API

CARTO fully supports PostGIS analytics via its SQL API. 

Check [this](https://carto.com/academy/courses/sql-postgis/postgis-in-cartodb) to have a sneak peak on the CARTO analytics capabilities via the SQL API or check the official [PostGIS docs](http://postgis.net/docs/manual-2.2/) to know more about it.

**Disclaimer**: You can run this queries from the SQL console within CARTO and visualize the results in real time, build dashboards, etc.

In [ ]:
# Let's start by knowing the PostGIS version supported by CARTO
POSTGIS_VERSION = "SELECT PostGIS_Version() as version;"

result = sql.send(POSTGIS_VERSION)

print(result["rows"][0]["version"])

In [ ]:
# How many tornados around 10KM of New York?
CITY_WKT = "POINT(-73.973694 40.731649)" # New York. To find coordinates of another city in the USA -> http://bboxfinder.com/
METERS = 10000

COUNT_TORNADOS_NEAR_CITY = "select count(*) as number_of_tornados from tornados where st_dwithin(the_geom::geography, st_geometryfromtext('" + CITY_WKT + "', 4326)::geography, " + str(METERS) + ")"

result = sql.send(COUNT_TORNADOS_NEAR_CITY)

print_result(result)

In [ ]:
# Give me all the information
GET_TORNADOS_NEAR_CITY = "select * from tornados where st_dwithin(the_geom::geography, st_geometryfromtext('" + CITY_WKT + "', 4326)::geography, " + str(METERS) + ")"
result = sql.send(GET_TORNADOS_NEAR_CITY)
print_result(result)

In [ ]:
# Which was the most damaging tornado in the USA
MAX_DAMAGE_TORNADO = "select cartodb_id from tornados order by damage desc limit 1"
result = sql.send(MAX_DAMAGE_TORNADO)
id_tornado = result['rows'][0]["cartodb_id"]
print(id_tornado)

In [ ]:
# Which area affected?
BUFFER_SIZE_METERS = 1000
BUFFER_AROUND_MAX_DAMAGE_TORNADO = "select st_astext(st_buffer(the_geom::geography, " + str(BUFFER_SIZE_METERS) + ")) as damage_zone from tornados where cartodb_id = " + str(id_tornado)
result = sql.send(BUFFER_AROUND_MAX_DAMAGE_TORNADO)
print(result['rows'][0]["damage_zone"])

# COPY AND PASTE THE RESULTING POLYGON HERE -> https://arthur-e.github.io/Wicket/

In [ ]:
# export data as GeoJSON
import json

METERS = 100000
GET_TORNADOS_NEAR_CITY_AS_GEOJSON = "select * from tornados where st_dwithin(the_geom::geography, st_geometryfromtext('" + CITY_WKT + "', 4326)::geography, " + str(METERS) + ")"
result = sql.send(GET_TORNADOS_NEAR_CITY_AS_GEOJSON, format="geojson")
result_as_json = json.dumps(result, sort_keys=True, indent=4, separators=(',', ': '))
print(result_as_json)

# COPY AND PASTE THE OUTPUT HERE -> http://geojson.io/ 

In [ ]:
# play a bit with the SQL API
# add here other SQL queries, try updating, inserting or deleting rows, JOINING other datasets in your account, etc.








#### Batch SQL

In [ ]:
from carto.sql import BatchSQLClient

LIST_OF_SQL_QUERIES = [__WRITE_HERE_LONG_QUERIES_LIKE_UPDATES_OR_INSERTS__]

batchSQLClient = BatchSQLClient(auth_client)
createJob = batchSQLClient.create(LIST_OF_SQL_QUERIES)

print(createJob['job_id'])

In [ ]:
readJob = batchSQLClient.read(createJob['job_id'])

In [ ]:
print(readJob)

### Maps API

In [ ]:
EXISTING_POINT_DATASET="tornados"
NAMED_MAP_AUTH_TOKEN = "ugfhfdyt5re8treu"
NAMED_MAP_QUERY = "select * from {dataset}".format(dataset=EXISTING_POINT_DATASET)
NAMED_MAP_NAME = "python_sdk_test_map1"
NAMED_MAP_DEFINITION = {
    "version": "0.0.1",
    "name": NAMED_MAP_NAME,
    "auth": {
        "method": "token",
        "valid_tokens": [
            NAMED_MAP_AUTH_TOKEN
        ]
    },
    "placeholders": {
        "color": {
            "type": "css_color",
            "default": "red"
        }
    },
    "layergroup": {
        "version": "1.0.1",
        "layers": [
            {
                "type": "http",
                "options": {
                    "urlTemplate": "http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png"
                }
            },
            {
                "type": "cartodb",
                "options": {
                    "cartocss_version": "2.1.1",
                    "cartocss": "#layer { marker-fill: <%= color %>; }",
                    "sql": NAMED_MAP_QUERY
                }
            }
        ]
    },
    "view": {
        "zoom": 4,
        "center": {
            "lng": 0,
            "lat": 0
        },
        "bounds": {
            "west": -45,
            "south": -45,
            "east": 45,
            "north": 45
        }
    }
}

In [ ]:
from carto.maps import NamedMapManager, NamedMap
import json

named_map_manager = NamedMapManager(auth_client)
named_map = NamedMap(named_map_manager.client)

# Create named map
named = named_map_manager.create(template=NAMED_MAP_DEFINITION)

In [ ]:
print(named)

In [ ]:
from carto.maps import NamedMapManager, NamedMap
import json

# write the path to a local file with a JSON named map template
JSON_TEMPLATE = { 
    "color": "#ff6600" 
}

# write here the ID of the named map
NAMED_MAP_ID = "python_sdk_test_map1"

# write here the token you set to the named map when created
NAMED_MAP_TOKEN = "ugfhfdyt5re8treu"

named_map_manager = NamedMapManager(auth_client)
named_map = named_map_manager.get(NAMED_MAP_ID)

named_map.instantiate(JSON_TEMPLATE, NAMED_MAP_TOKEN)

In [ ]:
# a tile around New York -> see http://www.maptiler.org/google-maps-coordinates-tile-bounds-projection/ to know the z,x,y values for any tile in the world
# named_map.get_tile_url(X, Y, Z, layer_id="all")
tile_new_york = named_map.get_tile_url(75, 96, 8, layer_id="all")
print(tile_new_york)

- tile: 8, 75, 96
<img src="PASTE_HERE_THE_URL_OF_THE_PREVIOUS_STEP_AND_RUN_THIS_CELL" />

In [ ]:
from carto.maps import NamedMapManager, NamedMap

# write here the ID of the named map
NAMED_MAP_ID = "python_sdk_test_map1"

# get the named map created
named_map = named_map_manager.get(NAMED_MAP_ID)

# update named map
named_map.view = None
named_map.save()

# delete named map
#named_map.delete()

# list all named maps
named_maps = named_map_manager.all()

# let's delete all the previously created named maps
# for n in named_maps:
    # n.delete()

In [ ]:
# PLAY AROUND WITH THE NAMED MAP TEMPLATE
# FOR EXAMPLE TRY A DIFFERENT QUERY (one of the analysis we made in the SQL API section), CHANGE THE CARTOCSS OR ADD INTERACTION
# NOTE: IF THE NAMED MAP IS NOT UPDATED, TRY TO DELETE THE PREVIOUS VERSION OR USE A DIFFERENT NAME



## Examples

Try by yourself some of the `carto-python` [examples](https://github.com/CartoDB/carto-python/tree/master/examples)